In [2]:
import gspread
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
from tqdm import tqdm as tqdm
from pathlib import Path

In [3]:
gc = gspread.oauth(http_client=gspread.BackOffHTTPClient)

In [4]:
file_urls = {
    annot :f"https://docs.google.com/spreadsheets/d/{file_id}" for annot, file_id in [
        ("melanie", "1sQMKc1M__VSbG9LNqvKHtrIkY2bCc0Z4IQ8ZAfwHJQg"),
        ("elisa", "1lrRshWRZySYv11ZAM2ulNO1_aELuU04ql5EyrsbRPwI"),
        ("stanislas", "1WHiEUThe_eZ9Ae2byWKg5YAK-ZpdCVgFUKqOfnDi6Zw"),
    ]
}

In [5]:
def load_worksheet(sheet):
    sample_id = int(sheet.get("A1")[0][0])
    data = pd.DataFrame(sheet.get_all_records(head=2))
    data = data.iloc[:, 1:]
    data["A ne pas évaluer"] = data["A ne pas évaluer"].replace({'TRUE': True, 'FALSE': False}).astype(bool)
    data = data[~data["A ne pas évaluer"]]
    data.index = data.loc[:, "Code critère"]
    data = data.loc[:, "Respecté ?"]
    data["sample_id"] = sample_id
    
    return data

def load_eval_spreadsheet(file_url, annotator):
    file = gc.open_by_url(file_url)
    sheets = file.worksheets()
    df = pd.DataFrame([load_worksheet(s) for s in sheets])
    df = df.set_index("sample_id")
    df.loc[:, "annotator"] = annotator
    df = df.set_index("annotator", append=True)
    return df

def load_all(file_urls, local_dir, save=True, load=True):
    dfs = []
    for annot, url in file_urls.items():
        path = local_dir / f"{annot}.csv"
        if load and path.exists():
            df = pd.read_csv(path)
        else:
            df = load_eval_spreadsheet(url, annot)
        if save:
            df.to_csv(path)
        dfs.append(df)
            
    return pd.concat(dfs)

In [6]:
gen_df = pd.read_json("./selected_samples.json")
eval_df = load_all(file_urls,Path("./annot_tables/").resolve())
df = gen_df.merge(eval_df, left_index=True, right_on='sample_id')

In [7]:
df.to_csv(Path("./annot_tables/")/"results.csv")

In [8]:
df

,example_id,model,source,target,prediction,dataset,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_id,...,I1,I2,CA1,CA2,CA3,CA4,CA5,CA6,CA7,CA8
0,72,mbarthez-lora,"PAIX ET INDÉPENDANCE. Climat, guerres, crises ...",Paix et indépendance. Les pays européens doive...,PAIX ET INDÉPENDANCE L’Europe de la paix et de...,POLITIC,0.0,0,0,144,...,Non,Non Applicable,Bon,Très bon,Bon,Bon,Non,Non,Non,Bon
0,72,mbarthez-lora,"PAIX ET INDÉPENDANCE. Climat, guerres, crises ...",Paix et indépendance. Les pays européens doive...,PAIX ET INDÉPENDANCE L’Europe de la paix et de...,POLITIC,0.0,0,0,144,...,Non,Non Applicable,Satisfaisant,Bon,Bon,Satisfaisant,Non,Non,Non,Satisfaisant
0,72,mbarthez-lora,"PAIX ET INDÉPENDANCE. Climat, guerres, crises ...",Paix et indépendance. Les pays européens doive...,PAIX ET INDÉPENDANCE L’Europe de la paix et de...,POLITIC,NaN,0,0,144,...,Non,Non,Satisfaisant,Bon,Bon,Bon,Non,Non,Non,Bon
1,49,mistral-lora,Un intrus dans la maison – Maow… cela faisait ...,Panique dans la maison Jules et Bou ont bien d...,Un intrus dans la maison Jules et Bou se révei...,FALC,1.0,1,1,99,...,Non Applicable,Non Applicable,Satisfaisant,Bon,Satisfaisant,Mauvais,Oui,Non,Oui,Mauvais
1,49,mistral-lora,Un intrus dans la maison – Maow… cela faisait ...,Panique dans la maison Jules et Bou ont bien d...,Un intrus dans la maison Jules et Bou se révei...,FALC,1.0,1,1,99,...,Non,Non Applicable,Satisfaisant,Bon,Satisfaisant,Mauvais,Oui,Non,Non,Satisfaisant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,68,mistral-lora,"POUR LA JEUNESSE, LA CULTURE ET L’AVENIR DE NO...",Pour les jeunes et nos enfants. • Nous aurons ...,"POUR LA JEUNESSE, LA CULTURE ET L’AVENIR DE NO...",POLITIC,58.0,58,58,137,...,Non,Non Applicable,Bon,Très bon,Très bon,Très bon,Oui,Non,Non,Mauvais
58,68,mistral-lora,"POUR LA JEUNESSE, LA CULTURE ET L’AVENIR DE NO...",Pour les jeunes et nos enfants. • Nous aurons ...,"POUR LA JEUNESSE, LA CULTURE ET L’AVENIR DE NO...",POLITIC,NaN,58,58,137,...,Non,Non Applicable,Bon,Bon,Bon,Bon,Oui,Non,Non,Mauvais
59,72,mistral-lora,"PAIX ET INDÉPENDANCE. Climat, guerres, crises ...",Paix et indépendance. Les pays européens doive...,La souveraineté est notre meilleure arme. La s...,POLITIC,59.0,59,59,145,...,Non,Non,Satisfaisant,Bon,Satisfaisant,Satisfaisant,Non,Oui,Non,Satisfaisant
59,72,mistral-lora,"PAIX ET INDÉPENDANCE. Climat, guerres, crises ...",Paix et indépendance. Les pays européens doive...,La souveraineté est notre meilleure arme. La s...,POLITIC,59.0,59,59,145,...,Non,Non Applicable,Très bon,Très bon,Satisfaisant,Bon,Non,Non,Non,Satisfaisant


In [9]:
criteria = ['CI3', 'CI4', 'CI5', 'CI6', 'CI8', 'CPM1',
       'CPM2', 'CPM3', 'CPM4', 'CPM5', 'CPM6', 'CPM7', 'CPM8', 'CPM9', 'CPM10',
       'CPM11', 'CPM12', 'CPM13', 'CPM14', 'CPM15', 'CPM16', 'CPM17', 'CPM18',
       'CPM19', 'CPM20', 'CPM21', 'I1', 'I2', 'CA1', 'CA2', 'CA3', 'CA4',
       'CA5', 'CA6', 'CA7', 'CA8']

# Melting the DataFrame
df_melted = df.melt(id_vars=['example_id', 'model', 'source', 'target', 'prediction', 
                              'dataset', 'sample_id', 'annotator'],
                     value_vars=criteria,
                     var_name='criteria',
                     value_name='value')

In [14]:
themes_crit = {
    "Choix de l'information": ['CI3', 'CI4', 'CI5', 'CI6', 'CI8'],
    "Choix des mots": ['CPM2', 'CPM3', 'CPM4',
       'CPM5', 'CPM6', 'CPM8', 'CPM9',  'CPM16', 'CPM17', 'CPM18', 'CPM19',
       'CPM20', 'CPM21'],
    "Construction des phrases": [
        'CPM1', "CPM7", "CPM10", "CPM11", "CPM12", 'CPM13', 'CPM14','CPM15',
    ],
    "Illustrations": ['I1', 'I2'],
    "Qualité globales": ['CA1', 'CA2', 'CA3', 'CA4',
       'CA5', 'CA6', 'CA7', 'CA8'],
}
crit_themes = {
    crit: k for k, v in themes_crit.items() for crit in v
}
type_crit = {
    "ETR": ['CI3', 'CI4', 'CI5', 'CI6', 'CI8', 'CPM1',
       'CPM2', 'CPM3', 'CPM4', 'CPM5', 'CPM6', 'CPM7', 'CPM8', 'CPM9', 'CPM10',
       'CPM11', 'CPM12', 'CPM13', 'CPM14', 'CPM15', 'CPM16', 'CPM17', 'CPM18',
       'CPM19', 'CPM20', 'CPM21', 'I1', 'I2'],
    "global quality": ['CA1', 'CA2', 'CA3', 'CA4',
       'CA5', 'CA6', 'CA7', 'CA8']
}
crit_type = {
    crit: k for k, v in type_crit.items() for crit in v
}
df_melted["criteria_theme"] = df_melted["criteria"].replace(crit_themes)
df_melted["criteria_type"] = df_melted["criteria"].replace(crit_type)
df_melted["annotator"] = df_melted["annotator"].replace({"elisa": "coder_1", "melanie": "coder_2", "stanislas":"coder_3"})

In [15]:
# df_melted = df_melted.pivot(index=['example_id', 'model', 'source', 'target', 'prediction', 
#                                    'dataset', 'sample_id', 'criteria', 'criteria_theme', 'criteria_type'],
#                            columns='annotator',
#                            values='value').reset_index()

In [16]:
crit_to_score = {
    'CI3': 2,
    'CI4': 6,
    'CI5': 2,
    'CI6': 2,
    'CI8': 2,
    'CPM1': 6,
    'CPM2': 6,
    'CPM3': 6,
    'CPM4': 2,
    'CPM5': 6,
    'CPM6': 4,
    'CPM7': 2,
    'CPM8': 2,
    'CPM9': 2,
    'CPM10': 4,
    'CPM11': 2,
    'CPM12': 4,
    'CPM13': 4,
    'CPM14': 4,
    'CPM15': 2,
    'CPM16': 2,
    'CPM17': 2,
    'CPM18': 2,
    'CPM19': 2,
    'CPM20': 2,
    'CPM21': 2,
    'I1': 4,
    'I2': 4
}
df_melted["criteria_score"] = df_melted["criteria"].replace(crit_to_score)

In [17]:
df_melted.to_csv(Path("./annot_tables/")/"results_melted.csv")